In [15]:
# Importing libraries
import pandas as pd

# preparing al jazeera corpus

In [16]:
# Importing corpus
df = pd.read_csv('aljazeera_data.csv')

In [17]:
#removing empty lines
df.dropna()

,title,date,url,sentence
0,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,South Korea’s football association accused the...
1,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,The South Korean football association (KFA) ha...
2,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,The Seoul-based KFA’s letter to the Asian Foot...
3,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,"The game, which was cast into media darkness b..."
4,South Korea wants North punished over football...,2019-10-18,https://www.aljazeera.com/sports/2019/10/18/so...,"Upon their return, South Korean players said t..."
...,...,...,...,...
87481,Erdogan open to meeting al-Assad but not to wi...,2023-07-17,https://www.aljazeera.com/news/2023/7/17/erdog...,While tensions between the two countries remai...
87482,Erdogan open to meeting al-Assad but not to wi...,2023-07-17,https://www.aljazeera.com/news/2023/7/17/erdog...,The restoration of ties with Damascus quickene...
87483,Erdogan open to meeting al-Assad but not to wi...,2023-07-17,https://www.aljazeera.com/news/2023/7/17/erdog...,"On Sunday, Iraq’s prime minister held talks wi..."
87484,Erdogan open to meeting al-Assad but not to wi...,2023-07-17,https://www.aljazeera.com/news/2023/7/17/erdog...,The decision to allow Syria back into the Arab...


In [18]:
# removing contentless Al Jazeera boilerplate 
df.drop(df[df['sentence'].str.contains('Follow Al Jazeera English:')| df['sentence'].str.contains('Al Jazeera English:')].index, inplace=True)

In [19]:
# removing incorrect formated lines
try:
    # Überprüfe und ersetze ungültige Datumsangaben
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # Selektiere Zeilen mit ungültigen Datumsangaben
    invalid_rows = df[df['date'].isna()]

    # Überprüfe, ob ungültige Zeilen gefunden wurden
    if not invalid_rows.empty:
        print(f"{len(invalid_rows)} fehlerhafte Zeilen gefunden und entfernt.")
        # Entferne Zeilen mit NaT in der Spalte 'date'
        df = df.dropna(subset=['date'])
    else:
        print("Alle Datumsangaben korrekt.")
except Exception as e:
    print(f"Fehler beim Überprüfen der Datumsangaben: {e}")

293 fehlerhafte Zeilen gefunden und entfernt.


In [20]:
# adding coloumn year
df['year'] = pd.to_datetime(df['date']).dt.year

In [21]:
# remove liveblog entries without usefull content
df_check = df.groupby(['title','year','url'])['sentence'].agg(lambda x: ' '.join(str(val) for val in x if pd.notna(val))).reset_index()
df_check.drop(df_check[df_check['url'].str.contains('https://www.aljazeera.com/sports/liveblog')|df_check['url'].str.contains('https://www.aljazeera.com/news/liveblog')].index, inplace=True)

In [22]:
# adding relevance scores to secure that articels are related to our topic 
# keywords were determined by building a semantic field for the term 'fifa'.
# As semantic field "fifa", "world_cup", and "qatar" are chose. Fifa is extended 
# with the two presidents during the considered period of time.
# To ensure that the articels really contain news related to the fifa world cup in qatar, 
# each of these terms should be contained.

def calculate_score_fifa(text):
    keywords = ['fifa', 'blatter', 'infantino']
    # Konvertiere Text und Keywords in Kleinbuchstaben für case-sensitive Suche
    text_lower = text.lower()
    score = sum(text_lower.count(keyword) for keyword in keywords)
    return score

def calculate_score_world_cup(text):
    keywords = ['world cup']
    # Konvertiere Text und Keywords in Kleinbuchstaben für case-sensitive Suche
    text_lower = text.lower()
    score = sum(text_lower.count(keyword) for keyword in keywords)
    return score

def calculate_score_qatar(text):
    keywords = ['qatar']
    # Konvertiere Text und Keywords in Kleinbuchstaben für case-sensitive Suche
    text_lower = text.lower()
    score = sum(text_lower.count(keyword) for keyword in keywords)
    return score

# Konvertiere alle Zeichen in der Spalte 'Text' in Kleinbuchstaben
df_check['sentence'] = df_check['sentence'].str.lower()

# Scores für Tehemnzugehörigkeit und Länge berechnen
df_check['score_relevance_fifa'] = df_check['sentence'].apply(calculate_score_fifa)
df_check['score_relevance_qatar'] = df_check['sentence'].apply(calculate_score_qatar)
df_check['score_relevance_world_cup'] = df_check['sentence'].apply(calculate_score_world_cup)
df_check['score_article_length'] = df_check['sentence'].str.len()

#Ergebnis-Output zu Evaluation der notwendigen thresholds
output_check = df_check[['url', 'score_relevance_fifa', 'score_relevance_qatar','score_article_length','score_relevance_world_cup']].reset_index(drop=True)
#output_check.to_csv('output_check.csv', index=False)

# the threshold of the relevant terms is set to at least 1 appearance. 
# But for the longer articles its two. This decision was made by analyzing articles in the "output_check.csv" above.
aljazeera_filtered_df = df_check[(df_check['score_article_length'] < 2000) & (df_check['score_relevance_fifa'] > 0) & (df_check['score_relevance_qatar'] > 0) & (df_check['score_relevance_world_cup'] > 0) |
                 (df_check['score_article_length'] >= 2000) & (df_check['score_article_length'] < 25000) & (df_check['score_relevance_fifa'] > 1) & (df_check['score_relevance_qatar'] > 1) & (df_check['score_relevance_world_cup'] > 1)]

aljazeera_filtered_df.reset_index(drop=True, inplace=True)

aljazeera_filtered_df.rename(columns={'sentence': 'text'}, inplace=True)

aljazeera_filtered_df = aljazeera_filtered_df[['title', 'year', 'text']]
aljazeera_filtered_df['source'] = 'aljazeera'

#display(aljazeera_filtered_df)

C:\Users\OPEF\AppData\Local\Temp\ipykernel_50504\357486251.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aljazeera_filtered_df.rename(columns={'sentence': 'text'}, inplace=True)


# preparing guardian corpus

In [23]:
df_guardian = pd.read_csv('guardian_data_unprepared.csv')
#display(df_guardian)

In [24]:
df_guardian.dropna()

,webtitle,apiUrl,publisheddate,Text
0,Revealed: star football pundits promote World ...,https://www.theguardian.com/football/2022/nov/...,20.11.2022,Spirit of rules on use of celebrities in adver...
1,Fifa ethics chief Michael Garcia calls for gre...,https://www.theguardian.com/football/2014/oct/...,13.10.2014,Garcia has delivered his report on 2018 and 2...
2,World Cup 2022: news and buildup as the knocko...,https://www.theguardian.com/football/live/2022...,03.12.2022,Brazil suffer double injury blow going into th...
3,England’s Harry Kane may abandon ‘OneLove’ arm...,https://www.theguardian.com/football/2022/nov/...,20.11.2022,England’s players will take the knee before th...
4,Welcome to Lusail: Lego-city of the gods and o...,https://www.theguardian.com/football/2022/nov/...,27.11.2022,Andy Warhol would love the venue for the World...
...,...,...,...,...
1751,UK parliament calls on Fifa to support exiled ...,https://www.theguardian.com/football/2023/jun/...,20.06.2023,More than 100 parliamentarians have written to...
1752,Jesús Ferreira scores four as USA breeze past ...,https://www.theguardian.com/football/2022/jun/...,11.06.2022,Jesús Ferreira tied the United States’ interna...
1753,"The Guide #29: From All 4 to Netflix, the one ...",https://www.theguardian.com/culture/2022/apr/0...,08.04.2022,In this week’s newsletter: not sure what to st...
1754,World Cup 2022 briefing: Louis van Gaal is a m...,https://www.theguardian.com/football/2022/dec/...,09.12.2022,Louis van Gaal’s Dutch team may not play beaut...


In [25]:
#removing incorrect formated lines
try:
    # Überprüfe und ersetze ungültige Datumsangaben
    df_guardian['publisheddate'] = pd.to_datetime(df_guardian['publisheddate'], errors='coerce')

    # Selektiere Zeilen mit ungültigen Datumsangaben
    invalid_rows = df_guardian[df_guardian['publisheddate'].isna()]

    # Überprüfe, ob ungültige Zeilen gefunden wurden
    if not invalid_rows.empty:
        print(f"{len(invalid_rows)} fehlerhafte Zeilen gefunden und entfernt.")
        # Entferne Zeilen mit NaT in der Spalte 'date'
        df_guardian = df_guardian.dropna(subset=['publisheddate'])
    else:
        print("Alle Datumsangaben korrekt.")
except Exception as e:
    print(f"Fehler beim Überprüfen der Datumsangaben: {e}")

Alle Datumsangaben korrekt.


C:\Users\OPEF\AppData\Local\Temp\ipykernel_50504\1348653984.py:4: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_guardian['publisheddate'] = pd.to_datetime(df_guardian['publisheddate'], errors='coerce')


In [26]:
#Adding coloumn year
df_guardian['year'] = pd.to_datetime(df_guardian['publisheddate']).dt.year
#display(df_guardian)
df_guardian = df_guardian[df_guardian['year'] != 2024]

In [27]:
# adding relevance scores to secure that articels are related to our topic 
# keywords were determined by building a semantic field for the term 'fifa'.
# As semantic field "fifa", "world_cup", and "qatar" are chose. Fifa is extended 
# with the two presidents during the considered period of time.
# To ensure that the articels really contain news related to the fifa world cup in qatar, 
# each of these terms should be contained.

def calculate_score_fifa(text):
    keywords = ['fifa', 'world cup', 'blatter', 'infantino']
    # Konvertiere Text und Keywords in Kleinbuchstaben für case-sensitive Suche
    text_lower = text.lower()
    score = sum(text_lower.count(keyword) for keyword in keywords)
    return score

def calculate_score_world_cup(text):
    keywords = ['world cup']
    # Konvertiere Text und Keywords in Kleinbuchstaben für case-sensitive Suche
    text_lower = text.lower()
    score = sum(text_lower.count(keyword) for keyword in keywords)
    return score

def calculate_score_qatar(text):
    keywords = ['qatar']
    # Konvertiere Text und Keywords in Kleinbuchstaben für case-sensitive Suche
    text_lower = text.lower()
    score = sum(text_lower.count(keyword) for keyword in keywords)
    return score

# Konvertiere alle Zeichen in der Spalte 'Text' in Kleinbuchstaben
df_guardian['Text'] = df_guardian['Text'].str.lower()

# Scores für Tehemnzugehörigkeit und Länge berechnen
df_guardian['score_relevance_fifa'] = df_guardian['Text'].apply(calculate_score_fifa)
df_guardian['score_relevance_qatar'] = df_guardian['Text'].apply(calculate_score_qatar)
df_guardian['score_relevance_world_cup'] = df_guardian['Text'].apply(calculate_score_world_cup)
df_guardian['score_article_length'] = df_guardian['Text'].str.len()

#Ergebnis-Output zu Evaluation der notwendigen thresholds
output_check_guardian = df_guardian[['apiUrl', 'score_relevance_fifa', 'score_relevance_qatar','score_article_length','score_relevance_world_cup']].reset_index(drop=True)
#output_check_guardian.to_csv('output_check_guardian.csv', index=False)

# the threshold of the relevant terms is set to at least 1 appearance. 
# But for the longer articles its two. This decision was made by analyzing articles in the "output_check_guardian.csv" above.
guardian_filtered_df = df_guardian[(df_guardian['score_article_length'] < 2000) & (df_guardian['score_relevance_fifa'] > 0) & (df_guardian['score_relevance_qatar'] > 0) & (df_guardian['score_relevance_world_cup'] > 0)|
                 (df_guardian['score_article_length'] >= 2000) & (df_guardian['score_article_length'] < 25000) & (df_guardian['score_relevance_fifa'] > 1) & (df_guardian['score_relevance_qatar'] > 1)& (df_guardian['score_relevance_world_cup'] > 1)]

guardian_filtered_df.reset_index(drop=True, inplace=True)

guardian_filtered_df.rename(columns={'webtitle': 'title'}, inplace=True)
guardian_filtered_df.rename(columns={'Text': 'text'}, inplace=True)

guardian_filtered_df = guardian_filtered_df[['title', 'year', 'text']]
guardian_filtered_df['source'] = 'guardian'
#guardian_filtered_df.to_csv('guardian_filtered_prepared_df.csv', index=False)


C:\Users\OPEF\AppData\Local\Temp\ipykernel_50504\183462396.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guardian_filtered_df.rename(columns={'webtitle': 'title'}, inplace=True)
C:\Users\OPEF\AppData\Local\Temp\ipykernel_50504\183462396.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guardian_filtered_df.rename(columns={'Text': 'text'}, inplace=True)


# combining both corpora

In [28]:
complete_filtered_corpus_df = pd.concat([aljazeera_filtered_df, guardian_filtered_df], ignore_index=True)
complete_filtered_corpus_df.to_csv('complete_filtered_corpus.csv', index=False)